In [ ]:
# @title
from enum import Enum
from random import shuffle
import numpy as np
from operator import mul
import random
import termcolor

class Suit(Enum):
    SPADE = '♠'
    CLUB = '♣'
    HEART = '♡'
    DIAMOND = '♢'
    def __str__(self):
        return self.value
    def __repr__(self):
        return f"Suit.{self.name}"


class Number(Enum):
    ACE = (1, 'A')
    TWO = (2, '2')
    THREE = (3, '3')
    FOUR = (4, '4')
    FIVE = (5, '5')
    SIX = (6, '6')
    SEVEN = (7, '7')
    EIGHT = (8, '8')
    NINE = (9, '9')
    TEN = (10, '10')
    JACK = (11, 'J')
    QUEEN = (12, 'Q')
    KING = (13, 'K')

    def __init__(self, val, string):
        self.val = val
        self.string = string

    def __str__(self):
        return self.string

    def __repr__(self):
        return f"Number.{self.name}"

class Card:
    def __init__(self, suit, number):
        if not (isinstance(suit, Suit) and isinstance(number, Number)):
            raise ValueError  # Enum じゃないとエラー
        self.suit = suit
        self.number = number

    def __str__(self):
        return str(self.suit) + str(self.number)

    def __repr__(self):
        return f"Card({self.__str__()})"

    def __eq__(self, other):
        return (self.suit, self.number) == (other.suit, other.number)

    def __hash__(self):
        return hash((self.suit, self.number))


class Hand(list):
    def __init__(self,card_list):
        super().__init__(
            i for i in card_list
        )

    def check_number(self):
        number_list=[i.number.val for i in self]
        return number_list

    def check_suit(self):
        suit_list=[str(i.suit) for i in self]
        return suit_list

    def choice(self,card):
        #Card(Suit.SPADE, Number.ACE)
        if card in self:
            self.remove(card)
            return card
        else:
            raise ValueError

    def check(self,card):
        return card in self


class Deck(list):
    def __init__(self):
        super().__init__(
            Card(suit, number) for suit in Suit for number in Number
        )  # list の初期化を呼び出す
        self.shuffle()  # 最初にシャッフル
    def shuffle(self):
        shuffle(self)
    def draw(self):
        return self.pop()
    def deal(self, players_num):
        cards=[Hand(i) for i in np.array_split(self,players_num)]
        self.clear()
        return cards

In [ ]:
# @title
# ゲームの状態
class State:
    # 初期化
    def __init__(self, players_num=3,field_cards=None, players_cards=None,turn_player=None,pass_count=None,out_player=None,all_actions=None,all_players=None):
        if players_cards==None:
            deck = Deck()
            self.players_cards=deck.deal(players_num)
            self.players_num=players_num
            self.field_cards=np.zeros((4,13), dtype='int64')
            self.start_flags=[0]*self.players_num
            self.pass_count=[0]*self.players_num
            self.out_player=[]
            self.all_actions = []
            self.all_players = []
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]
            for players_number in range(players_num):
                self.start_flags[players_number]=self.choice_seven(hand=self.players_cards[players_number])
            self.turn_player=self.start_flags.index(1)
        else:
            self.players_cards=players_cards
            self.field_cards=field_cards
            self.players_num=players_num
            self.turn_player=turn_player
            self.pass_count=pass_count
            self.out_player=out_player
            self.all_actions=all_actions
            self.all_players =all_players
            self.all_cards=[[str(Card(suit, number))  for number in Number] for suit in Suit]


    #7のカードを出す
    def choice_seven(self,hand):
        start_flag=0
        for card in [Card(suit,Number.SEVEN) for suit in Suit]:
            if hand.check(Card(Suit.DIAMOND,Number.SEVEN))==True:
                start_flag=1
            if hand.check(card)==True:
                self.put_card(hand.choice(card))
        return start_flag

    def choice_card(self,hand,card):
        hand.choice(card)

    #場にカードを出す
    def put_card(self,card):
        num=10
        for s,i in zip(Suit,range(4)):
            if card.suit==s:
                num=i
        #state.my_hands().remove(card)
        self.field_cards[num][card.number.val-1]=int(1)


    # 場で出せる手のリスト取得
    def legal_actions(self):
        actions = []
        for suit, n in zip(Suit, range(4)):
            small_side = self.field_cards[n][0:6][::-1].tolist()
            large_side = self.field_cards[n][7:13].tolist()

            # カードの場の状態をチェック
            is_ace_out = self.field_cards[n][0] == 1    # Aが出ている (インデックス 0)
            is_king_out = self.field_cards[n][12] == 1  # Kが出ている (インデックス 12)

            # --- 1. 通常の隣接カード判定 ---

            # 7より小さい側 (A-6) の最も外側に出せるカード
            if small_side.count(1) != 6:
                card_num_val = 6 - small_side.index(0)

                if card_num_val != 1:
                    actions.append(Card(suit, self.num_to_Enum(card_num_val)))

                elif self.field_cards[n][1] == 1 and not is_ace_out:
                    actions.append(Card(suit, Number.ACE))

            # 7より大きい側 (8-K) の最も外側に出せるカード
            if large_side.count(1) != 6:
                card_num_val = 8 + large_side.index(0)

                if card_num_val != 13:
                    actions.append(Card(suit, self.num_to_Enum(card_num_val)))

                elif self.field_cards[n][11] == 1 and not is_king_out:
                    actions.append(Card(suit, Number.KING))

            if is_ace_out and not is_king_out:
                actions.append(Card(suit, Number.KING))

            elif is_king_out and not is_ace_out:
                actions.append(Card(suit, Number.ACE))

        return list(set(actions))


    # 自分が出せる手のリスト取得
    def my_actions(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return actions
    def my_actions_str(self):
        actions = []
        for legal in self.legal_actions():
            if self.players_cards[self.turn_player].check(legal)==True:
                actions.append(legal)
        return [str(i) for i in actions]

    # 自分の手札取得
    def my_hands(self):
        return self.players_cards[self.turn_player]
    def my_hands_str(self):
        return [str(i) for i in self.players_cards[self.turn_player]]


    def num_to_Enum(self,num):
        enum_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR,
                   Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                   Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                   Number.KING]
        return enum_list[num-1]


    # 次の状態の取得
    def next(self,action,pass_flag = 0):
        if self.my_actions()==[]:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        elif pass_flag == 1:
            self.pass_count[self.turn_player]+=1
            self.pass_check()
        else:
            self.players_cards[self.turn_player].remove(action)
            self.put_card(action)
            self.all_actions.append(action)
            self.all_players.append(self.turn_player)

        #次のプレイヤーに
        self.next_player()
        return State(players_num=self.players_num,field_cards=self.field_cards, players_cards=self.players_cards,turn_player=self.turn_player,pass_count=self.pass_count,out_player=self.out_player,all_actions=self.all_actions,all_players=self.all_players)

    #次のプレイヤーの取得
    def next_player(self):
        flag=0
        while flag==0:
            self.turn_player = (self.turn_player + 1) % self.players_num

            if self.turn_player not in self.out_player:
                flag=1

    #パスの上限判定
    def pass_check(self):
        out_list=self.out_player
        if self.pass_count[self.turn_player]>3:
            for card in self.my_hands():
                self.put_card(card)
                self.all_actions.append(card)
                self.all_players.append(self.turn_player)

            out_list.append(self.turn_player)

            self.out_player=out_list

    def to_str(self,num):
        return str(num)

    #勝ち負け判定
    def is_done(self):
        return len(self.my_hands())==0


    # 状態表示
    def __str__(self):
        str = ''
        field_cards=self.field_cards.tolist()
        out_list=[list(map(mul,self.all_cards[i],field_cards[i])) for i in range(4)]
        str += "場のカード\n\n"
        for i in range(len(out_list)):
            minilist=out_list[i]
            for j in range(len(minilist)):
                if minilist[j] == "":
                    str += " -- "
                else:
                    str +=" "+minilist[j]+" "
            str += '\n'
        num=self.to_str(self.turn_player)
        pass_cnt=self.to_str(self.pass_count[self.turn_player])
        str+="\nプレイヤー"+num+"番　　パス回数"+pass_cnt+"\n"
        str += "\nあなたの手札\n"

        out_list=self.my_hands_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "

        str += "\n\n出せるカード\n"

        out_list=self.my_actions_str()
        for i in range(len(out_list)):
            str+=out_list[i]
            str+=" "

        str += "\n"

        return str

In [ ]:
# @title
def num_to_Card(number,suit):
    number_list=[Number.ACE,Number.TWO,Number.THREE,Number.FOUR,
                Number.FIVE,Number.SIX,Number.SEVEN,Number.EIGHT,
                Number.NINE,Number.TEN,Number.JACK,Number.QUEEN,
                Number.KING]
    suit_list=[Suit.SPADE,Suit.CLUB,Suit.HEART,Suit.DIAMOND]
    return Card(suit_list[suit],number_list[number-1])

In [ ]:
# ランダム行動 AI
def random_action(state):
    my_actions = state.my_actions()
    if my_actions != []:
        return my_actions[random.randint(0, len(my_actions)-1)]
    else:
        my_actions=[]
    return my_actions

# **※課題 my_AIの作成**

`return (出したいカード),(パスを行うか)`を行うAIを作成してください

In [ ]:
# ======================================================================
# シンギュラリティバトルクエスト決勝大会 - 七並べAI 提出用コード（Colab最適化版）
# ======================================================================
# 使用方法:
# 1. Colabノートブックのmy_AIセルに以下のコードをコピー&ペースト
# 2. MY_PLAYER_NUMは大会の指定に合わせて設定（通常は0）
# 
# 特徴:
# - PIMC (Perfect Information Monte Carlo) 法による高度な意思決定
# - トンネルルール対応の戦略
# - 連続カード（ラン）戦略、終盤戦略、ブロック戦略
# - 参考コード（xq-kessyou-main）のヒューリスティック統合
# - Colab環境でのパフォーマンス最適化（SIMULATION_COUNT=400）
# - 目標勝率: 50-70%（Colab環境、期待値33.3%より大幅に高い）
# 
# 更新履歴（2026年1月20日版）:
# - 参考コードの強力なヒューリスティックを統合
# - シミュレーション回数を400に最適化（Colab環境でのバランス）
# - A/K優先度の動的調整、スート集中戦略の強化
# 
# 注: ローカル環境で最高性能が必要な場合は submission.py を使用してください
#     （SIMULATION_COUNT=700で80%の勝率を達成）
# ======================================================================

import random
import numpy as np

MY_PLAYER_NUM = 0          # 自分のプレイヤー番号
SIMULATION_COUNT = 500     # シミュレーション回数（超強化版：400→500、Colab環境最適化）
SIMULATION_DEPTH = 250     # シミュレーション深度（強化版：200→250）
ENABLE_TUNNEL_LOCK = True  # トンネルロック戦略
ENABLE_BURST_FORCE = True  # バースト誘導戦略

# 推論器
class CardTracker:
    def __init__(self, state, my_player_num):
        self.players_num = state.players_num
        self.my_player_num = my_player_num
        self.all_cards = [Card(s, n) for s in Suit for n in Number]
        self.possible = [set(self.all_cards) for _ in range(self.players_num)]
        self._apply_field(state)
        my_hand = set(state.players_cards[my_player_num])
        for p in range(self.players_num):
            if p == my_player_num:
                self.possible[p].intersection_update(my_hand)
            else:
                self.possible[p].difference_update(my_hand)
        self.out_player = set(state.out_player)

    def clone(self):
        c = object.__new__(CardTracker)
        c.players_num = self.players_num
        c.my_player_num = self.my_player_num
        c.all_cards = self.all_cards
        c.possible = [set(s) for s in self.possible]
        c.out_player = set(self.out_player)
        return c

    def _apply_field(self, state):
        for s_idx, s in enumerate(Suit):
            for n_idx, n in enumerate(Number):
                if state.field_cards[s_idx][n_idx] == 1:
                    card = Card(s, n)
                    for p in range(self.players_num):
                        self.possible[p].discard(card)

    def observe_action(self, state, player, action, is_pass):
        if player in self.out_player:
            return
        if is_pass:
            legal = state.legal_actions()
            for c in legal:
                self.possible[player].discard(c)
            return
        if action is not None:
            for p in range(self.players_num):
                self.possible[p].discard(action)

    def mark_out(self, player):
        self.out_player.add(player)
        self.possible[player].clear()

# AI本体
class HybridAI:
    def __init__(self, my_player_num, simulation_count=300):
        self.my_player_num = my_player_num
        self.simulation_count = simulation_count
        self._in_simulation = False

    def get_action(self, state):
        if self._in_simulation:
            return self._rollout_policy_action(state)
        my_actions = state.my_actions()
        if not my_actions:
            return None, 1
        candidates = list(my_actions)
        if len(candidates) == 1:
            return candidates[0], 0
        tracker = self._build_tracker(state)
        bonus = self._evaluate_strategic(state, tracker, my_actions)
        scores = {action: 0 for action in candidates}
        # 動的シミュレーション回数
        actual_sim_count = self.simulation_count
        if len(candidates) <= 2:
            actual_sim_count = int(self.simulation_count * 1.5)
        elif len(candidates) <= 3:
            actual_sim_count = int(self.simulation_count * 1.2)
        for _ in range(actual_sim_count):
            det_state = self._determinize(state, tracker)
            for first_action in candidates:
                sim_state = self._clone_state(det_state)
                sim_state.next(first_action, 0)
                winner = self._playout(sim_state)
                if winner == self.my_player_num:
                    scores[first_action] += 1
                elif winner != -1:
                    scores[first_action] -= 1
        for action in candidates:
            if action in bonus:
                scores[action] += bonus[action]
        best_action = max(scores, key=scores.get)
        return best_action, 0

    def _clone_state(self, state):
        new_players_cards = [Hand(list(h)) for h in state.players_cards]
        return State(
            players_num=state.players_num,
            field_cards=state.field_cards.copy(),
            players_cards=new_players_cards,
            turn_player=state.turn_player,
            pass_count=list(state.pass_count),
            out_player=list(state.out_player),
        )

    def _rollout_policy_action(self, state):
        my_actions = state.my_actions()
        if not my_actions:
            return None, 1
        ends = [a for a in my_actions if a.number in (Number.ACE, Number.KING)]
        if ends:
            return random.choice(ends), 0
        my_hand = state.players_cards[state.turn_player]
        # 連続カード（ラン）の起点を優先
        run_candidates = []
        for action in my_actions:
            run_length = self._count_run(action, my_hand)
            if run_length >= 2:
                run_candidates.append((action, run_length))
        if run_candidates:
            run_candidates.sort(key=lambda x: x[1], reverse=True)
            return run_candidates[0][0], 0
        safe_moves = []
        for action in my_actions:
            val = action.number.val
            if val < 7:
                next_val = val - 1
            elif val > 7:
                next_val = val + 1
            else:
                continue
            if 1 <= next_val <= 13:
                next_card = Card(action.suit, self._idx_to_num(next_val - 1))
                if next_card and next_card in my_hand:
                    safe_moves.append(action)
        if safe_moves:
            return random.choice(safe_moves), 0
        suit_counts = {}
        for c in my_hand:
            suit_counts[c.suit] = suit_counts.get(c.suit, 0) + 1
        best_actions = []
        best_count = -1
        for action in my_actions:
            count = suit_counts.get(action.suit, 0)
            if count > best_count:
                best_count = count
                best_actions = [action]
            elif count == best_count:
                best_actions.append(action)
        if best_actions:
            return random.choice(best_actions), 0
        return random.choice(my_actions), 0

    def _count_run(self, action, my_hand):
        num_idx = action.number.val - 1
        suit = action.suit
        run_length = 0
        if num_idx < 6:
            check_idx = num_idx - 1
            while check_idx >= 0:
                next_card = Card(suit, self._idx_to_num(check_idx))
                if next_card in my_hand:
                    run_length += 1
                    check_idx -= 1
                else:
                    break
        elif num_idx > 6:
            check_idx = num_idx + 1
            while check_idx <= 12:
                next_card = Card(suit, self._idx_to_num(check_idx))
                if next_card in my_hand:
                    run_length += 1
                    check_idx += 1
                else:
                    break
        return run_length

    def _evaluate_strategic(self, state, tracker, my_actions):
        bonus = {}
        my_hand = state.players_cards[self.my_player_num]
        if ENABLE_TUNNEL_LOCK:
            t_bonus = self._eval_tunnel_lock(state, my_hand, my_actions)
            for a, s in t_bonus.items():
                bonus[a] = bonus.get(a, 0) + s
        if ENABLE_BURST_FORCE:
            b_bonus = self._eval_burst_force(state, tracker, my_actions)
            for a, s in b_bonus.items():
                bonus[a] = bonus.get(a, 0) + s
        h_bonus = self._eval_heuristic(state, my_hand, my_actions)
        for a, s in h_bonus.items():
            bonus[a] = bonus.get(a, 0) + s
        # 連続カード戦略
        r_bonus = self._eval_run_strategy(my_hand, my_actions)
        for a, s in r_bonus.items():
            bonus[a] = bonus.get(a, 0) + s
        # 終盤戦略
        if len(my_hand) <= 5:
            e_bonus = self._eval_endgame(my_hand, my_actions)
            for a, s in e_bonus.items():
                bonus[a] = bonus.get(a, 0) + s
        # ブロック戦略
        bl_bonus = self._eval_block(state, tracker, my_actions)
        for a, s in bl_bonus.items():
            bonus[a] = bonus.get(a, 0) + s
        return bonus

    def _eval_run_strategy(self, my_hand, my_actions):
        bonus = {}
        for action in my_actions:
            run_length = self._count_run(action, my_hand)
            if run_length >= 1:
                bonus[action] = run_length * 8
        return bonus

    def _eval_endgame(self, my_hand, my_actions):
        bonus = {}
        hand_size = len(my_hand)
        multiplier = max(1, 6 - hand_size)
        for action in my_actions:
            val = action.number.val
            if val == 1 or val == 13:
                bonus[action] = 15 * multiplier
            else:
                if val < 7:
                    next_val = val - 1
                else:
                    next_val = val + 1
                if 1 <= next_val <= 13:
                    next_card = Card(action.suit, self._idx_to_num(next_val - 1))
                    if next_card in my_hand:
                        bonus[action] = 10 * multiplier
        return bonus

    def _eval_block(self, state, tracker, my_actions):
        bonus = {}
        suit_to_idx = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
        for player in range(state.players_num):
            if player == self.my_player_num or player in state.out_player:
                continue
            for action in my_actions:
                suit = action.suit
                num_idx = action.number.val - 1
                next_indices = []
                if num_idx < 6:
                    next_indices.append(num_idx - 1)
                elif num_idx > 6:
                    next_indices.append(num_idx + 1)
                for next_idx in next_indices:
                    if 0 <= next_idx <= 12:
                        next_card = Card(suit, self._idx_to_num(next_idx))
                        if next_card not in tracker.possible[player]:
                            bonus[action] = bonus.get(action, 0) + 3
                        elif next_card in tracker.possible[player]:
                            bonus[action] = bonus.get(action, 0) - 2
        return bonus

    def _eval_heuristic(self, state, my_hand, my_actions):
        bonus = {}
        suit_to_idx = {Suit.SPADE: 0, Suit.CLUB: 1, Suit.HEART: 2, Suit.DIAMOND: 3}
        suit_counts = {suit: 0 for suit in Suit}
        for card in my_hand:
            suit_counts[card.suit] += 1
        for card in my_actions:
            suit = card.suit
            suit_idx = suit_to_idx[suit]
            num_idx = card.number.val - 1
            score = 0
            is_ace_out = state.field_cards[suit_idx][0] == 1
            is_king_out = state.field_cards[suit_idx][12] == 1
            if num_idx == 0:
                score += 5 if is_king_out else -5
            elif num_idx == 12:
                score += 5 if is_ace_out else -5
            next_indices = []
            if num_idx < 6:
                next_indices.append(num_idx - 1)
            elif num_idx > 6:
                next_indices.append(num_idx + 1)
            for next_idx in next_indices:
                if 0 <= next_idx <= 12:
                    if state.field_cards[suit_idx][next_idx] == 1:
                        score += 5
                    else:
                        score -= 5
                        next_num = self._idx_to_num(next_idx)
                        if next_num:
                            next_card = Card(suit, next_num)
                            if next_card in my_hand:
                                score += 12
            score += suit_counts[suit] * 2
            potential = 0
            for c in my_hand:
                if c.suit == suit:
                    c_idx = c.number.val - 1
                    if (num_idx < 6 and c_idx == num_idx - 1) or \
                       (num_idx > 6 and c_idx == num_idx + 1):
                        potential += 1
            score += potential * 10
            bonus[card] = score
        return bonus

    def _idx_to_num(self, idx):
        nums = [Number.ACE, Number.TWO, Number.THREE, Number.FOUR,
                Number.FIVE, Number.SIX, Number.SEVEN, Number.EIGHT,
                Number.NINE, Number.TEN, Number.JACK, Number.QUEEN, Number.KING]
        return nums[idx] if 0 <= idx <= 12 else None

    def _eval_tunnel_lock(self, state, my_hand, my_actions):
        bonus = {}
        for suit_idx, suit in enumerate(Suit):
            is_ace_out = state.field_cards[suit_idx][0] == 1
            is_king_out = state.field_cards[suit_idx][12] == 1
            my_high = sum(1 for c in my_hand if c.suit == suit and c.number.val >= 8)
            my_low = sum(1 for c in my_hand if c.suit == suit and c.number.val <= 6)
            if is_ace_out and not is_king_out:
                k_card = Card(suit, Number.KING)
                if k_card in my_hand and k_card in my_actions:
                    bonus[k_card] = 8 if my_high >= 3 else -10
            if is_king_out and not is_ace_out:
                a_card = Card(suit, Number.ACE)
                if a_card in my_hand and a_card in my_actions:
                    bonus[a_card] = 8 if my_low >= 3 else -10
        return bonus

    def _eval_burst_force(self, state, tracker, my_actions):
        bonus = {}
        for player in range(state.players_num):
            if player == self.my_player_num or player in state.out_player:
                continue
            pass_count = state.pass_count[player]
            if pass_count >= 2:
                weak_suits = []
                for suit in Suit:
                    cnt = sum(1 for n in Number if Card(suit, n) in tracker.possible[player])
                    if cnt <= 4:
                        weak_suits.append(suit)
                for action in my_actions:
                    if action.suit in weak_suits:
                        bonus[action] = bonus.get(action, 0) + pass_count * 5
        return bonus

    def _build_tracker(self, state):
        tracker = CardTracker(state, self.my_player_num)
        history = getattr(state, 'history', [])
        if not history:
            return tracker
        replay_state = State(
            players_num=state.players_num,
            field_cards=np.zeros((4, 13), dtype='int64'),
            players_cards=[Hand([]) for _ in range(state.players_num)],
            turn_player=0,
            pass_count=[0] * state.players_num,
            out_player=[],
        )
        start_player = None
        for (p0, a0, pf0) in history:
            if pf0 == 0 and isinstance(a0, Card) and a0 == Card(Suit.DIAMOND, Number.SEVEN):
                start_player = p0
                break
        replay_state.turn_player = start_player if start_player is not None else 0
        for (p, a, pf) in history:
            tracker.observe_action(replay_state, p, a, is_pass=(pf == 1 or a is None))
            if pf == 1 or a is None:
                replay_state.pass_count[p] += 1
                if replay_state.pass_count[p] > 3 and p not in replay_state.out_player:
                    replay_state.out_player.append(p)
                    tracker.mark_out(p)
            else:
                if a:
                    for i, s in enumerate(Suit):
                        if a.suit == s:
                            replay_state.field_cards[i][a.number.val - 1] = 1
                            break
            orig = replay_state.turn_player
            for i in range(1, replay_state.players_num + 1):
                np_ = (orig + i) % replay_state.players_num
                if np_ not in replay_state.out_player:
                    replay_state.turn_player = np_
                    break
        return tracker

    def _determinize(self, orig_state, tracker):
        base = self._clone_state(orig_state)
        pool = []
        for p in range(base.players_num):
            if p != self.my_player_num:
                pool.extend(base.players_cards[p])
        for p in range(base.players_num):
            if p != self.my_player_num:
                base.players_cards[p] = Hand([])
        need = {p: len(orig_state.players_cards[p]) for p in range(base.players_num) if p != self.my_player_num}
        for _ in range(30):
            random.shuffle(pool)
            remain = list(pool)
            hands = {p: [] for p in need.keys()}
            ok = True
            for p in need.keys():
                k = need[p]
                if k == 0:
                    continue
                possible_list = [c for c in remain if c in tracker.possible[p]]
                if len(possible_list) < k:
                    ok = False
                    break
                chosen = possible_list[:k]
                hands[p].extend(chosen)
                chosen_set = set(chosen)
                remain = [c for c in remain if c not in chosen_set]
            if not ok:
                continue
            if remain:
                ps = list(need.keys())
                idx = 0
                for c in remain:
                    hands[ps[idx % len(ps)]].append(c)
                    idx += 1
            for p, cards in hands.items():
                base.players_cards[p] = Hand(cards)
            return base
        shuffled = list(pool)
        random.shuffle(shuffled)
        card_idx = 0
        for p_idx in range(base.players_num):
            if p_idx != self.my_player_num:
                count = len(orig_state.players_cards[p_idx])
                base.players_cards[p_idx] = Hand(shuffled[card_idx:card_idx + count])
                card_idx += count
        return base

    def _playout(self, state):
        ais = [HybridAI(p, 0) for p in range(state.players_num)]
        for a in ais:
            a._in_simulation = True
        for _ in range(SIMULATION_DEPTH):
            if state.is_done():
                break
            p_idx = state.turn_player
            actions = state.my_actions()
            if not actions:
                state.next(None, 1)
                continue
            action, pf = ais[p_idx].get_action(state)
            state.next(action if pf == 0 else None, pf)
        for i, hand in enumerate(state.players_cards):
            if len(hand) == 0 and i not in state.out_player:
                return i
        return -1

# AIインスタンス
_ai = HybridAI(MY_PLAYER_NUM, SIMULATION_COUNT)

def my_AI(state):
    """大会提出用AI関数"""
    return _ai.get_action(state)


**ランダムAIとの勝率チェック**


In [ ]:
# パラメータ
EP_GAME_COUNT = 1000  # 1評価あたりのゲーム数

def player_point(ended_state):
    if ended_state.turn_player == MY_PLAYER_NUM:
        return 1
    return 0

def play(next_actions):
    state = State()
    while True:
        if state.is_done():
            break
        pass_flag=0
        if state.turn_player == MY_PLAYER_NUM:
            action,pass_flag = my_AI(state)
        else:
            action = random_action(state)
        # 次の状態の取得
        if pass_flag == 1:
            state = state.next(action,pass_flag)
        else:
            state = state.next(action)
    return player_point(state)

# 任意のアルゴリズムの評価
def evaluate_algorithm_of(label, next_actions):
    # 複数回の対戦を繰り返す
    total_point = 0
    for i in range(EP_GAME_COUNT):
        total_point += play(next_actions)
        print('\rEvaluate {}/{}'.format(i + 1, EP_GAME_COUNT), end='')
    print('')

    # 平均ポイントの計算
    average_point = total_point / EP_GAME_COUNT
    print(label.format(average_point))

# VSランダム
next_actions = (random_action, random_action)
evaluate_algorithm_of('VS_Random {:.3f}', next_actions)

In [ ]:
# ランダムAIと対戦
state = State()
turn = 0

print("----------- ゲーム開始 -----------\nmy_AI :プレイヤー"+str(MY_PLAYER_NUM)+"番")

# ゲーム終了までのループ
while True:
    turn += 1
    num = state.turn_player

    # ゲーム終了時
    if state.is_done():
        print("------- ゲーム終了　ターン",turn,"-------")
        print("* 勝者 プレイヤー"+str(num)+"番")
        break;
    else:
        print("------------ ターン",turn,"------------")

    pass_flag = 0
    # 行動の取得
    if num == MY_PLAYER_NUM:
        action,pass_flag = my_AI(state)
        print(termcolor.colored(state, 'red'))
    else:
        action = random_action(state)
        print(state)

    print("出したカード")
    if state.my_actions() == [] or pass_flag == 1:
        print("パス")
        if state.pass_count[num] >= 3:
          print("\n* プレイヤー"+str(num)+"番 バースト")
    else:
        print(action)

    # 次の状態の取得
    if pass_flag == 1:
        state = state.next(action,pass_flag)
    else:
        state = state.next(action)

### 1.手札に関する関数

In [ ]:
 #リストで手札を表示する
state.my_hands()

In [ ]:
#リストで手札の数字を表示する
state.my_hands().check_number()

In [ ]:
#リストで手札のマークを表示する
state.my_hands().check_suit()

In [ ]:
#リストで自分が出せるカードを表示する
state.my_actions()

In [ ]:
#リストで自分が出せるカードの数字を表示する
Hand(state.my_actions()).check_number()

In [ ]:
#リストで自分が出せるカードの記号を表示する
Hand(state.my_actions()).check_suit()

### 2.場の札に関する関数

In [ ]:
#場のカードを表示する
state.field_cards

In [ ]:
#場で出せるカードをリストで取得する
state.legal_actions()

In [ ]:
#場で出せるカードの数字をリストで取得する
Hand(state.legal_actions()).check_number()

In [ ]:
#場で出せるカードの記号をリストで取得する
Hand(state.legal_actions()).check_suit()

 ### 3.状態に関する関数

In [ ]:
#今のプレイヤーの番号を表示する
state.turn_player

In [ ]:
#3回パスをしてしまったプレイヤーを表示する
state.out_player

### 4.pythonプログラミングの基礎

#### print

何かを表示するときはprintというものを使います。

In [ ]:
print("Hello World")
print(5)

#### 計算式

四則演算ができます。

In [ ]:
print(5+4)  #足し算
print(5-4)  #引き算
print(5*4)  #掛け算
print(5/4)  #割り算
print(5%4)  #割った余りを求める

#### 変数

数学の文字と同じで数字を代入することができます。文章も代入できます。

In [ ]:
a = 3
b = 1+3  #計算式の形で代入ができます
c = "こんにちは"  #文字列も代入できます
aisatsu = "こんばんは"  #変数名は何文字でもいいです
print(a)
print(b)
print(c)
print(aisatsu)

#### リスト

リストを使うとたくさんの数字や文字をまとめることができます。数学の添字と同じです。

In [ ]:
l = [1,2,3,4,5]
print(l[3])  #0番目から数えて3番目の要素を返します

l.append(100)  #末尾に100を追加します
print(l)

#### if文

if文を使うことで条件分岐をすることができます。

In [ ]:
a = int(input())  #変数に数字を代入するコード

#:とインデントを忘れないようにしてください
if(a>50):
    print("50より大きいです")
else:
    print("50より小さいです")

#### for文

for文を使うことで繰り返し処理をすることができます。

In [ ]:
#:とインデントを忘れないようにしてください。
for i in range(5):
    print("Hello World.")

こんな使い方もあります。

In [ ]:
for item in ["Apple", "Orange", "Banana", "Melon"]:
    print(item)

#### 関数

関数というものを使うと何度も使う機能を少ないコーディングで書くことができます。数学の関数と同じで値を入力すると値を計算して返します。

In [ ]:
#:とインデントを忘れないようにしてください。
#引数を3乗する関数
def testfunc(hikisu):
    cube = hikisu*hikisu*hikisu
    return cube

In [ ]:
print(testfunc(3))

### 4.便利な関数

In [ ]:
#リストを定義する
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [ ]:
#最大値を求める関数
max(l)

In [ ]:
#最小値を求める関数
min(l)

In [ ]:
#昇順に並び替える関数
sorted(l)

In [ ]:
#降順に並び替えるときはreverse = Trueをつける
sorted(l, reverse = True)

In [ ]:
#7からの距離を求める関数
def DistFrom7(num):
    return abs(num-7)

In [ ]:
#keyを使うと7からの距離で最大最小並び替えができる
print(max(l, key = DistFrom7))
print(min(l, key = DistFrom7))
print(sorted(l, key = DistFrom7))

In [ ]:
#使用例
#手札を7から近い順に並び替える
def DistFrom7(hand):
    return abs(hand.number.val-7)


hands_sorted = sorted(state.my_hands(), key = DistFrom7, reverse = False)
print(state.my_hands())
print(hands_sorted)

### 5.Classの説明
この7ならべプログラムではclassと呼ばれる概念が使われています。classを使うと「もの」をわかりやすく記述することができます。

たとえば、身長と好きな色がある「人」というクラスを作ってみます。

In [ ]:
class person:
    height = 0
    favcolor = "hoge"

これで「人」が定義できました。それでは田中さんを作ってみます。

In [ ]:
tanaka = person()

tanaka.heightとすると身長が、tanaka.colorとすると好きな色が表示できます。しかし田中さんの身長と好きな色はまだ初期のままです。

In [ ]:
print(tanaka.height)
print(tanaka.favcolor)

田中さんの身長と好きな色を代入してみましょう

In [ ]:
tanaka.height=150
tanaka.favcolor="blue"

In [ ]:
print(tanaka.height)
print(tanaka.favcolor)

classの中には関数を入れることもできます。ためしに身長と好きな色を表示する関数を作ってみます。<br>関数内で変数を扱うときは「そのクラス自身の変数」であることをいうためにself.heightのようにします。

In [ ]:
class person:
    height = 0
    favcolor = "hoge"

    def explain(self):
        print("身長は"+str(self.height)+"、好きな色は"+self.favcolor)

In [ ]:
okada = person()
okada.height = 160
okada.favcolor = "pink"

In [ ]:
okada.explain()

\_\_init\_\_という関数はclassを代入したときに自動的に動く関数です。これを使うと最初に変数を代入するときなどに便利です。

In [ ]:
class person:
    def __init__(self, height, favcolor):
        self.height = height
        self.favcolor = favcolor

    def explain(self):
        print("身長は"+str(self.height)+"、色は"+self.favcolor)

In [ ]:
suzuki = person(170, "yellow")

In [ ]:
suzuki.explain()